In [1]:
import requests, json
from tqdm import tqdm
from bs4 import BeautifulSoup
import urllib.parse as parse
import pandas as pd

In [2]:
def articlesAdrsLstGen(keywords):
  adrs_lst = []

  for page in tqdm(range(1, 3002, 10)):
    try:
      if len(adrs_lst) == 1100:
        break
      else:
        headers = {
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36',
        }
        url = 'https://search.naver.com/search.naver?'
        qs = 'where=news&sm=tab_pge&query=러시아%20한국%20경제&sort=1&photo=0&field=0&pd=0&ds=&de=&cluster_rank=54&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start=1'  
        params = dict(parse.parse_qsl(qs))
        params['cluster_rank'] = ''
        params['query'] = keywords
        params['start'] = page
        resp = requests.get(url, headers=headers, params=params)
        soup = BeautifulSoup(resp.content, 'lxml')
        adrs = soup.select('div.news_info > div.info_group > a')
        for item in adrs:
          if len(adrs_lst) == 1100:
            break
          elif 'https://news.naver.com/main/read.naver?' in item['href'] and item['href'] not in adrs_lst:
            adrs_lst.append(item['href'])
    except Exception as e:
      print()
      print(e)
      print(item['href'])
      continue
  return adrs_lst

# 'div.news_wrap.api_ani_send > div > div.news_info > div.info_group > a
# '#sp_nws45 > div.news_wrap.api_ani_send > div > div.news_info > div.info_group > a:nth-child(3)'
# '#sp_nws50 > div > div > div.news_info > div.info_group > a:nth-child(3)'

def articlesTableGen(adrs_lst):
  articles = {
    '등록일시': [],
    '제목': [],
    '내용': [],
    '주소': [],
  }

  for adrs in tqdm(adrs_lst):
    if len(articles['제목']) == 1000:
      break
    else:
      try:
        headers = {
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36',
        }
        url = adrs
        resp = requests.get(url, headers=headers)
        soup = BeautifulSoup(resp.content, 'lxml')
        get_title = soup.select('#articleTitle')
        get_time = soup.select('#main_content > div.article_header > div.article_info > div > span.t11')
        get_content = soup.select('#articleBodyContents')
        articles['등록일시'].append(get_time[0].text)
        articles['제목'].append(get_title[0].text)
        articles['내용'].append(get_content[0].text)
        articles['주소'].append(adrs)
      except Exception as e:
        print()
        print(e)
        print('오류주소', adrs)
        continue
  table = pd.DataFrame(articles)
  return table


In [3]:
lst = articlesAdrsLstGen('러시아 한국 경제')

 64%|██████▍   | 194/301 [04:14<02:20,  1.31s/it]


In [4]:
len(lst)

1100

In [5]:
len(set(lst))

1100

In [6]:
df = articlesTableGen(lst)

 91%|█████████ | 1000/1100 [09:35<00:57,  1.74it/s]


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   등록일시    1000 non-null   object
 1   제목      1000 non-null   object
 2   내용      1000 non-null   object
 3   주소      1000 non-null   object
dtypes: object(4)
memory usage: 31.4+ KB


In [8]:
df

,등록일시,제목,내용,주소
0,2022.04.14. 오전 11:38,수입 곡물 가격 상승 우려에 사료株 '고공행진',\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,https://news.naver.com/main/read.naver?mode=LS...
1,2022.04.14. 오전 11:37,"""韓기준금리 2.86%까지 오를 수도…가구당 이자 345만원 증가""",\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,https://news.naver.com/main/read.naver?mode=LS...
2,2022.04.14. 오전 11:33,한경연 “美 금리 동조 시 韓 기준금리 2.86%까지 인상”,\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,https://news.naver.com/main/read.naver?mode=LS...
3,2022.04.14. 오전 11:28,"""美 FOMC 빅스텝 예고…국내 기준금리 2.86%까지 상승 전망""",\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,https://news.naver.com/main/read.naver?mode=LS...
4,2022.04.14. 오전 11:20,물가·부채·성장 ‘세마리 토끼잡기’ 전속력 질주,\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,https://news.naver.com/main/read.naver?mode=LS...
...,...,...,...,...
995,2022.03.06. 오후 4:12,한국 경제 ‘저성장-고물가’ 심화 우려···“스태그플레이션도 배제 못해”,\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,https://news.naver.com/main/read.naver?mode=LS...
996,2022.03.06. 오후 3:27,금 2298.5t 쟁여둔 러시아...'금융핵폭탄'에도 '믿는구석' 있었다,\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,https://news.naver.com/main/read.naver?mode=LS...
997,2022.03.06. 오후 3:23,"""우크라 충격…韓경제 경기 재침체 가능성""",\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,https://news.naver.com/main/read.naver?mode=LS...
998,2022.03.06. 오후 1:18,우크라發 물가 충격 이달부터 현실화…스태그플레이션 우려,\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,https://news.naver.com/main/read.naver?mode=LS...


In [9]:
cond = df.duplicated()
df[cond].count()

등록일시    0
제목      0
내용      0
주소      0
dtype: int64

In [ ]:
df

In [11]:
df.to_csv('러시아 한국 경제.csv', index=False)